In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from megatron.neox_arguments import NeoXArgs
from megatron.training import setup_model_and_optimizer
from megatron.initialize import initialize_megatron
from megatron import mpu

import deepspeed

[2025-01-22 03:06:30,197] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/adam/.conda/envs/jax311/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/adam/.conda/envs/jax311/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/adam/.conda/envs/jax311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Unable to import Mamba kernels. Install them from our requirements/requirements-mamba.txt,     or directly from https://github.com/state-spaces/mamba
For s3 checkpointing, please install boto3 either using requirements/requirements-s3.txt or https://github.com/boto/boto3
For s3 checkpointing, please install hf_transfer either using requirements/requirements-s3.txt or https://github.com/huggingface/hf_transfer


In [3]:

YAML_PATH = "/mnt/hdd-0/tiny-pythia/ckpts/pythia-14m/global_step256/configs/pythia-14m.yml"
CHECKPOINT_PATH = "/mnt/hdd-0/tiny-pythia/ckpts/pythia-14m/" # mp_rank_00_model_states.pt

# 1. Load the configuration
args = NeoXArgs.from_ymls([YAML_PATH])

# Set the load path for the checkpoint and other necessary args
args.load = CHECKPOINT_PATH
args.vocab_size = 50304
args.padded_vocab_size = 50304

# Add fusion settings
args.scaled_upper_triang_masked_softmax_fusion = False
args.bias_gelu_fusion = False
args.masked_softmax_fusion = False

# Add MP settings
args.rank = 0
args.world_size = 1
args.model_parallel_size = 1
args.pipe_parallel_size = 1
args.num_stages = 1
args.num_layers_per_virtual_pipeline_stage = 1

# Add DeepSpeed settings
args.train_batch_size = 1
args.train_micro_batch_size_per_gpu = 1
args.zero_optimization = {"stage": 0}

print("\nSet args\n")

print("\nSetting up model parallel world size...\n")
mpu.set_model_parallel_world_size(1)  # Since you're using model parallel size of 1
print("\nSet model parallel world size\n")

print("\nSetting up model parallel rank...\n")
mpu.set_model_parallel_rank(0)  # Since you have rank 0
print("\nSet model parallel rank\n")

# Initialize distributed backend first
print("\nInitializing distributed...\n")
deepspeed.init_distributed()

# Then set up model parallel groups
print("\nSetting up model parallel...\n")
mpu.initialize_model_parallel(model_parallel_size=1)

# Initialize Megatron after MP setup
print("\nInitializing Megatron...\n")
initialize_megatron(args)

# Now load model and optimizer
print("\nSetting up model and optimizer...\n")
model, optimizer, lr_scheduler = setup_model_and_optimizer(args, iteration=256)

NeoXArgs.from_ymls() ['/mnt/hdd-0/tiny-pythia/ckpts/pythia-14m/global_step256/configs/pythia-14m.yml']

Set args


Setting up model parallel world size...


Set model parallel world size


Setting up model parallel rank...


Set model parallel rank


Initializing distributed...

[2025-01-22 03:06:33,455] [INFO] [comm.py:652:init_distributed] cdb=None
[2025-01-22 03:06:33,456] [INFO] [comm.py:667:init_distributed] Not using the DeepSpeed or dist launchers, attempting to detect MPI environment...


[2025-01-22 03:06:33,803] [INFO] [comm.py:718:mpi_discovery] Discovered MPI settings of world_rank=0, local_rank=0, world_size=1, master_addr=10.146.101.79, master_port=29500
[2025-01-22 03:06:33,804] [INFO] [comm.py:683:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl

Setting up model parallel...

> initializing model parallel with size 1

Initializing Megatron...

torch distributed is already initialized, skipping initialization ...
_initialize_distributed() model parallel is already initialized
[2025-01-22 03:06:33,809] [INFO] [checkpointing.py:1125:configure] Activation Checkpointing Information
[2025-01-22 03:06:33,809] [INFO] [checkpointing.py:1126:configure] ----Partition Activations False, CPU CHECKPOINTING False
[2025-01-22 03:06:33,810] [INFO] [checkpointing.py:1127:configure] ----contiguous Memory Checkpointing False with 6 total layers
[2025-01-22 03:06:33,810] [INFO] [checkpointing.py:1128:configure] ----Synchronization True
[2025-01-22 03:06:33,

Using /home/adam/.cache/torch_extensions/py311_cu124 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/adam/.cache/torch_extensions/py311_cu124/fused_adam/build.ninja...
/home/adam/.conda/envs/jax311/lib/python3.11/site-packages/torch/utils/cpp_extension.py:1964: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
Building extension module fused_adam...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.
Time to load fused_adam op: 0.0392146110534668 seconds
> learning rate decay style: cosine
DeepSpeed is enabled.
[2025-01-22 03:06:34,023] [INFO] [logging.py:128:log_dist] [Rank 0] DeepSpeed info: version=0.16.3, git-hash=unknown, git-branch=unknown
[2025-01-22 03:06:34,024] [INFO] [config.py:733:__init__] Config mesh_device None world_size = 1


Loading extension module fused_adam...


[2025-01-22 03:06:34,212] [INFO] [logging.py:128:log_dist] [Rank 0] DeepSpeed Flops Profiler Enabled: False
[2025-01-22 03:06:34,214] [INFO] [logging.py:128:log_dist] [Rank 0] Using client Optimizer as basic optimizer
[2025-01-22 03:06:34,214] [INFO] [logging.py:128:log_dist] [Rank 0] Removing param_group that has no 'params' in the basic Optimizer
[2025-01-22 03:06:34,215] [INFO] [logging.py:128:log_dist] [Rank 0] DeepSpeed Basic Optimizer = FusedAdam
[2025-01-22 03:06:34,216] [INFO] [logging.py:128:log_dist] [Rank 0] Creating fp16 optimizer with dynamic loss scale
[2025-01-22 03:06:34,229] [INFO] [logging.py:128:log_dist] [Rank 0] DeepSpeed Final Optimizer = FP16_Optimizer
[2025-01-22 03:06:34,230] [INFO] [logging.py:128:log_dist] [Rank 0] DeepSpeed using client LR scheduler
[2025-01-22 03:06:34,231] [INFO] [logging.py:128:log_dist] [Rank 0] DeepSpeed LR Scheduler = <megatron.learning_rates.AnnealingLR object at 0x7f6df40c3d10>
[2025-01-22 03:06:34,232] [INFO] [logging.py:128:log_dis

RuntimeError: Error(s) in loading state_dict for SequentialWrapper:
	Missing key(s) in state_dict: "sequential.2.mlp.linear1.weight", "sequential.2.mlp.linear1.bias", "sequential.2.mlp.linear2.weight", "sequential.2.mlp.linear2.bias", "sequential.3.mlp.linear1.weight", "sequential.3.mlp.linear1.bias", "sequential.3.mlp.linear2.weight", "sequential.3.mlp.linear2.bias", "sequential.4.mlp.linear1.weight", "sequential.4.mlp.linear1.bias", "sequential.4.mlp.linear2.weight", "sequential.4.mlp.linear2.bias", "sequential.5.mlp.linear1.weight", "sequential.5.mlp.linear1.bias", "sequential.5.mlp.linear2.weight", "sequential.5.mlp.linear2.bias", "sequential.6.mlp.linear1.weight", "sequential.6.mlp.linear1.bias", "sequential.6.mlp.linear2.weight", "sequential.6.mlp.linear2.bias", "sequential.7.mlp.linear1.weight", "sequential.7.mlp.linear1.bias", "sequential.7.mlp.linear2.weight", "sequential.7.mlp.linear2.bias". 
	Unexpected key(s) in state_dict: "sequential.2.attention.rotary_emb.inv_freq", "seq

# Scratch

In [ ]:
# import deepspeed
# import megatron.mpu as mpu

# # Initialize process group for DeepSpeed
# deepspeed.init_distributed(
#     # dist_backend='nccl',
#     # #distributed_port=12355,
#     # verbose=True
# )

# # Initialize MPU variables
# mpu.initialize_model_parallel(1)  # Set model parallel size to 1 for single GPU

[2025-01-22 02:20:21,798] [INFO] [comm.py:656:init_distributed] cdb=None
[2025-01-22 02:20:21,799] [INFO] [comm.py:671:init_distributed] Not using the DeepSpeed or dist launchers, attempting to detect MPI environment...


[2025-01-22 02:20:22,168] [INFO] [comm.py:722:mpi_discovery] Discovered MPI settings of world_rank=0, local_rank=0, world_size=1, master_addr=10.146.101.79, master_port=29500
[2025-01-22 02:20:22,169] [INFO] [comm.py:687:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl
> initializing model parallel with size 1
